# Data utilities
> A set of convenience functions for handling data

In [1]:
#| default_exp data_utils

In [2]:
#| export
import fastcore.test
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas
import os
import uuid
import tqdm
import matplotlib.pyplot as plt

In [3]:
#| export
dropped_items_warning=f"""Several items in `results` are not suitable for conversion to
a dataframe. This may be because they are not numpy arrays or because they
are not the same size as the other items. 

The following items were dropped: """
def results_to_dataframe_egt(results:dict, # A dictionary containing items from `ModelTypeEGT`.
                             suppress:bool=True, # Supress the dropped items warning
                            ):
    """Convert results to a dataframe, keeping only items which are valid for
    a dataframe to have."""
    flat_results = {k:v
                    for k,v in results.items()
                    if (isinstance(v, np.ndarray)
                        and not v.ndim > 1)}
    for i, strategy in enumerate(results.get('recurrent_states', 
                                             results.get('strategy_set', []))):
        if "ergodic" in list(results.keys()):
            flat_results[strategy + "_frequency"] = results['ergodic'][:,i]
    dropped_items = [k for k in results.keys() if k not in flat_results]
    if (len(dropped_items)>0 and not suppress):
        print(f"{dropped_items_warning} {dropped_items}")
    return pandas.DataFrame(flat_results)    

In [4]:
#| export
def process_dsair_data(data):
    """Process DSAIR model results dataframe."""
    data['pr'] = np.round(1 - data['p'].values, 2)
    data['s'] = np.round(data['s'].values, 2)
    return data

In [5]:
#| export

def run_all_simulations(param_list: list,
                        simulation_fn:callable=None,
                        plotting_fn:callable=None,
                        simulation_dir:str="",
                        plot_dir:str="",):
    """
    Iterate over each parameter dictionary, run the simulation, and save the results.

    Parameters:
    - param_list: A list of dictionaries, each containing a set of parameter values.
    """
    
    # Check if the output directory exists. If not, create it.
    if simulation_dir and not os.path.exists(simulation_dir):
        os.makedirs(simulation_dir)
    if plot_dir and not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        
    figs = []
    simulation_results = []
    # Construct a unique filename
    simulation_id = uuid.uuid4()
    for idx, parameters in tqdm(enumerate(param_list)):
        if simulation_fn is not None:
            df = simulation_fn(parameters)
            df["simulation_id"] = simulation_id
            df["model_id"] = idx
            # Save the dataframe to CSV
            filename = f"dataframe_{simulation_id}_{idx}.csv"
            filepath = os.path.join(simulation_dir, filename)
            df.to_csv(filepath, index=False)
            print(f"Saved file: {filepath}")
            simulation_results.append(df)
        if plotting_fn is not None:
            fig = plotting_fn(parameters)
            # Save the figure
            filename = f"plot_{simulation_id}_{idx}.png"
            filepath = os.path.join(plot_dir, filename)
            fig.savefig(filepath)
            print(f"Saved file: {filepath}")
            plt.close(fig)  # Close the figure to free up memory
            figs.append(fig)
        
    return figs, simulation_results

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()